# Инженерка

## Подготовка данных

Получаем типы физических объектов

In [1]:
import requests
import pandas as pd

res = requests.get('http://10.32.1.107:5300/api/v1/physical_object_types')
physical_objects_types = pd.DataFrame(res.json()).set_index('physical_object_type_id', drop=True)
# physical_objects_types.to_excel('pot.xlsx')
physical_objects_types.head()

,name,physical_object_function
physical_object_type_id,,
1,Рекреационная зона,None
2,Водный объект,None
3,Зеленая зона,None
4,Жилой дом,None
5,Здание,None


Типы индикаторов

In [2]:
res = requests.get('http://10.32.1.107:5300/api/v1/indicators_by_parent', {'get_all_subtree':True})
indicators = pd.DataFrame(res.json()).set_index('indicator_id', drop=True)
indicators.head()

,name_full,name_short,measurement_unit,level,list_label,parent_id,created_at,updated_at
indicator_id,,,,,,,,
261,Расчет обеспеченности,Расчет обеспеченности,None,1,5,NaN,2024-10-02T06:28:21.885235Z,2024-10-02T06:28:21.885235Z
34,Демография,Демография,None,1,2,NaN,2024-10-02T06:28:21.885235Z,2024-10-02T06:28:21.885235Z
46,Транспорт,Транспорт,None,1,3,NaN,2024-10-02T06:28:21.885235Z,2024-10-02T06:28:21.885235Z
135,Экология,Экология,None,1,6,NaN,2024-10-02T06:28:21.885235Z,2024-10-02T06:28:21.885235Z
178,Социальные риски,Социальные риски,None,1,8,NaN,2024-10-02T06:28:21.885235Z,2024-10-02T06:28:21.885235Z


Маппинг индикаторов и типов физ объектов

In [3]:
INDICATORS_POTS = {
  88: [],
  89: [21, 33, 34, 35],
  90: [38, 40],
  91: [37, 39],
  92: [],
  93: [],
}

In [4]:
for ind_id in INDICATORS_POTS.keys():
  print(indicators.loc[ind_id, 'name_full'])

Количество объектов инженерной инфраструктуры
Количество электростанций (ТЭС, АЭС и пр.)
Количество больших водозаборов
Количество очистительных сооружений
Количество крупных водохранилищ
Количество газораспределительных станций


In [46]:
import requests
import geopandas as gpd
import shapely
import json

URBAN_API = 'http://10.32.1.107:5300'
PAGE_SIZE = 10_000

def _get_physical_objects(region_id : int, pot_id : int, page : int, page_size : int = PAGE_SIZE):
  res = requests.get(f'{URBAN_API}/api/v1/territory/{region_id}/physical_objects_with_geometry', {
    'physical_object_type_id': pot_id,
    'page': page,
    'page_size': page_size,
  })
  return res.json()

def get_physical_objects(region_id : int, pot_id : int):
  page = 1
  results = []
  while True:
    res_json = _get_physical_objects(region_id, pot_id, page, page_size=PAGE_SIZE)
    results.extend(res_json['results'])
    if res_json['next'] is None:
      break
    page += 1
  for result in results:
    g = result['geometry']
    result['geometry'] = shapely.from_geojson(json.dumps(g))
  return gpd.GeoDataFrame(results)

In [45]:
import pandas as pd

def fetch_required_objects(region_id : int, pot_ids : list[int]):
  gdfs = [get_physical_objects(region_id, pot_id) for pot_id in pot_ids]
  return pd.concat(gdfs).set_geometry('geometry').set_crs(4326)

In [39]:
from townsnet.engineering.engineering_model import EngineeringObject

ENG_OBJ_TO_INDICATOR = {
  EngineeringObject.ENGINEERING_OBJECT : 88,
  EngineeringObject.POWER_PLANTS : 89,
  EngineeringObject.WATER_INTAKE : 90,
  EngineeringObject.WATER_TREATMENT : 91,
  EngineeringObject.WATER_RESERVOIR : 92,
  EngineeringObject.GAS_DISTRIBUTION : 93 
}

In [47]:
gdfs = {}
for eng_obj, ind_id in ENG_OBJ_TO_INDICATOR.items():
  pot_ids = INDICATORS_POTS[ind_id]
  if len(pot_ids) > 0:
    gdf = fetch_required_objects(1, pot_ids)
    gdfs[eng_obj] = gdf

In [20]:
import requests
import geopandas as gpd
import pandas as pd

URL = 'http://10.32.1.107:5300'

def get_region_territories(region_id : int) -> dict[int, gpd.GeoDataFrame]:
    res = requests.get(URL + '/api/v1/all_territories', {
        'parent_id': region_id,
        'get_all_levels': True
    })
    gdf = gpd.GeoDataFrame.from_features(res.json()['features'], crs=4326)
    # df = pd.json_normalize(gdf['territory_type']).rename(columns={
    #     'name':'territory_type_name'
    # })
    gdf = gdf.set_index('territory_id', drop=True)
    return {level:gdf[gdf['level'] == level] for level in set(gdf.level)}

In [21]:
adm_gdfs = get_region_territories(1)
last_key = list(gdfs.keys())[-1]
del gdfs[last_key]

## Работаем

In [48]:
from townsnet.engineering.engineering_model import EngineeringModel

em = EngineeringModel(gdfs)

In [86]:
def _aggregate(gdf : gpd.GeoDataFrame, units : gpd.GeoDataFrame):
  sjoin = gdf.sjoin(units[['geometry']], predicate='within')
  return sjoin.groupby('index_right').size()

def aggregate(self, units : gpd.GeoDataFrame) -> gpd.GeoDataFrame:
  units = units[['geometry']].copy()
  for eng_obj in list(EngineeringObject):
    if eng_obj in self.gdfs:
      gdf = self.gdfs[eng_obj]
      agg_gdf = _aggregate(gdf, units)
      units[eng_obj.value] = agg_gdf
      units[eng_obj.value] = units[eng_obj.value].fillna(0).astype(int)
    else:
      units[eng_obj.value] = 0
  return units

aggregate(em, adm_gdfs[3])

,geometry,Объект инженерной инфраструктуры,Электростанция,Водозабор,Водоочистительное сооружение,Водохранилище,Газораспределительная станция
territory_id,,,,,,,
2,"POLYGON ((34.32834 59.19564, 34.32777 59.19548...",0,0,0,0,0,0
10,"POLYGON ((28.98894 59.48069, 28.98604 59.48401...",0,0,0,0,0,0
18,"POLYGON ((32.85314 60.51175, 32.84899 60.50379...",0,0,0,0,0,0
34,"MULTIPOLYGON (((30.55161 59.96981, 30.55200 59...",0,0,0,0,0,0
54,"POLYGON ((28.99640 60.03638, 28.85880 60.04000...",0,0,0,0,0,0
67,"POLYGON ((29.63489 59.57073, 29.63372 59.57069...",0,0,0,0,0,0
85,"POLYGON ((28.19199 59.32665, 28.19187 59.32671...",0,0,0,0,0,0
97,"POLYGON ((31.94177 59.42479, 31.94327 59.42428...",0,0,0,0,0,0
104,"POLYGON ((31.91715 59.95987, 31.91722 59.95916...",0,0,0,0,0,0
